In [1]:
#import statements 
import glob
import random
from typing import List
from collections import defaultdict

import numpy as np
from numpy.random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from symusic import Score
from miditok import REMI, TokenizerConfig
from midi2audio import FluidSynth # Import library
from IPython.display import Audio, display

C:\Users\sammy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Task 1 <br>
This assignment focuses on symbolic music modeling. The goal is to train a model that learns a distribution \( p(x) \) over symbolic music data (e.g., MIDI)  specifically within the EDM genre. In addition it is capable of sampling new sequences from this learned distribution unconditionally. We will be using the LSTM model for this task. <br>

Get list of files for training/test sets

In [2]:
import os
import glob
print("CWD:", os.getcwd())
print("Train directory exists?", os.path.exists("./train"))
print("Train files (glob):", glob.glob("./train/*.midi"))

train_files = glob.glob("./train/*.midi")
test_files = glob.glob("./test/*.midi")

CWD: c:\Users\sammy\Downloads\cse 153_task1\cse153_task1
Train directory exists? True
Train files (glob): ['./train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_02_Track02_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_03_Track03_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav.midi', './train\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_05_Track05_wav.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--1.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--2.midi', './train\\MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3.midi', './train\\MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_2009_01_WAV.midi', './train\\MIDI-Unprocessed_01_R1_2009_01-04_ORIG_MID--AUDIO_01_R1_2009_01_R1_

Construct a PyTorch Dataset

In [3]:
class MIDIDataset(Dataset):
    def __init__(self, file_paths: List[str], tokenizer):
        self.tokenizer = tokenizer
        self.file_paths = file_paths
        
    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        midi = Score(self.file_paths[idx])
        MAX_LEN = 512  # or even 512 to be safe
        tokens = self.tokenizer(midi)
        tokens = tokens[:MAX_LEN]
        # Return as dictionary to match training function expectations
        return {'input_ids': torch.tensor(tokens, dtype=torch.long)}

Configure the Tokenizer in order to be use to 

In [4]:
config = TokenizerConfig(
    num_velocities=32,           # Classical dynamics (was 1)
    use_chords=True,            # Essential for harmony (was False)
    use_programs=False,         # Piano only (was True)
    use_time_signatures=True,   # Classical changes time sigs
    use_rests=True,             # Important in classical
)
tokenizer = REMI(config)
tokenizer.train(vocab_size=1500, files_paths=train_files)
tokenizer.save("tokenizer.json")

Define PyTorch datasets and dataloaders

In [5]:
from torch.nn.utils.rnn import pad_sequence
from miditok.pytorch_data import DatasetMIDI, DataCollator
dataset = MIDIDataset(train_files, tokenizer)


train_dataset = DatasetMIDI(
    files_paths=train_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
test_dataset = DatasetMIDI(
    files_paths=test_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
print(f"# Train files loaded: {len(train_dataset)}")
print(f"# Test files loaded: {len(test_dataset)}")

collator = DataCollator(tokenizer.pad_token_id)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collator)


# Train files loaded: 938
# Test files loaded: 105


LSTM Model<br>

In [6]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, num_layers=2, dropout=0.3, bidirectional=False):
        super(MusicRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
            bidirectional=bidirectional
        )

        # If bidirectional, output dim doubles
        rnn_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        self.layer_norm = nn.LayerNorm(rnn_output_dim)
        self.fc = nn.Linear(rnn_output_dim, vocab_size)

    def forward(self, x, hidden=None):
        """
        x: (batch_size, seq_length)
        hidden: hidden states for LSTM
        """
        x = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        out, hidden = self.rnn(x, hidden)  # (batch_size, seq_length, rnn_output_dim)
        
        out = self.layer_norm(out)          # stabilize activations
        out = self.fc(out)                  # (batch_size, seq_length, vocab_size)

        return out, hidden

Training<br>

In [7]:
def train(model, train_loader, val_loader, vocab_size, num_epochs=10, lr=1e-4, device='cpu'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # --------- Training ---------
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = batch['input_ids'].to(device)  # (batch_size, seq_length)

            inputs = batch[:, :-1]
            targets = batch[:, 1:]

            optimizer.zero_grad()
            outputs, _ = model(inputs)
            outputs = outputs.reshape(-1, vocab_size)
            targets = targets.reshape(-1)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --------- Validation ---------
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch['input_ids'].to(device)

                inputs = batch[:, :-1]
                targets = batch[:, 1:]

                outputs, _ = model(inputs)
                outputs = outputs.reshape(-1, vocab_size)
                targets = targets.reshape(-1)

                loss = criterion(outputs, targets)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


# Example usage
if __name__ == "__main__":
    vocab_size = tokenizer.vocab_size
    embedding_dim = 256
    hidden_dim = 512
    num_layers = 2

    model = MusicRNN(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.3,
    bidirectional=False   # try True if you want to experiment with bidirectional context
)

    train(model, train_loader, test_loader, vocab_size)

Epoch 1/10 | Train Loss: 6.4398 | Val Loss: 5.9706
Epoch 2/10 | Train Loss: 5.8206 | Val Loss: 5.7486
Epoch 3/10 | Train Loss: 5.5928 | Val Loss: 5.5155
Epoch 4/10 | Train Loss: 5.2966 | Val Loss: 5.1142
Epoch 5/10 | Train Loss: 4.8946 | Val Loss: 4.7733
Epoch 6/10 | Train Loss: 4.6610 | Val Loss: 4.6088
Epoch 7/10 | Train Loss: 4.5303 | Val Loss: 4.5059
Epoch 8/10 | Train Loss: 4.4355 | Val Loss: 4.4309
Epoch 9/10 | Train Loss: 4.3591 | Val Loss: 4.3625
Epoch 10/10 | Train Loss: 4.2933 | Val Loss: 4.3050


Sampling<br>

In [11]:

def sample_top_k(model, start_token, max_length=100, temperature=1.0, k=10, device='cpu'):
    model = model.to(device)
    model.eval()

    generated = [start_token]
    input_token = torch.tensor([[start_token]], device=device)  # (1, 1)
    hidden = None

    for _ in range(max_length):
        output, hidden = model(input_token, hidden)  # output: (1, 1, vocab_size)
        logits = output[:, -1, :] / temperature       # (1, vocab_size)

        # Top-k filtering
        top_k_logits, top_k_indices = torch.topk(logits, k)
        top_k_probs = F.softmax(top_k_logits, dim=-1)

        next_token_idx = torch.multinomial(top_k_probs, 1).item()
        next_token = top_k_indices[0, next_token_idx].item()

        generated.append(next_token)

        # Stop if EOS or PAD tokens are generated (you can adjust ids accordingly)
        if next_token == tokenizer["EOS_None"] or next_token == tokenizer.pad_token_id:
            break

        input_token = torch.tensor([[next_token]], device=device)

    return generated

start_token = tokenizer.special_tokens_ids[1]  # Typically BOS token

generated_sequence = sample_top_k(model, start_token, max_length=1024, temperature=1.0, k=10)
print("Generated token sequence:")
print(generated_sequence)


Generated token sequence:
[1, 410, 370, 360, 204, 63, 433, 205, 51, 460, 206, 44, 460, 209, 41, 469, 210, 56, 452, 211, 41, 446, 215, 32, 418, 215, 35, 418, 217, 37, 419, 219, 39, 445, 422, 37, 423, 190, 25, 423, 194, 44, 424, 196, 37, 452, 198, 39, 428, 200, 27, 433, 201, 58, 452, 203, 56, 445, 204, 56, 478, 207, 51, 445, 208, 54, 452, 211, 46, 445, 212, 56, 478, 214, 34, 477, 215, 51, 446, 219, 58, 445, 220, 51, 445, 466, 56, 478, 195, 34, 445, 197, 58, 445, 198, 58, 452, 199, 46, 446, 200, 58, 448, 205, 34, 448, 207, 27, 452, 360, 206, 39, 445, 208, 34, 486, 209, 34, 423, 211, 39, 448, 212, 44, 486, 216, 39, 452, 219, 36, 425, 220, 34, 425, 435, 39, 478, 194, 36, 448, 199, 34, 445, 203, 58, 486, 207, 36, 478, 41, 450, 39, 446, 41, 475, 48, 475, 36, 453, 36, 475, 48, 475, 36, 471, 361, 202, 60, 505, 48, 446, 48, 478, 53, 475, 53, 475, 60, 477, 41, 476, 53, 478, 360, 214, 41, 448, 58, 481, 36, 481, 53, 475, 360, 219, 58, 478, 46, 446, 435, 58, 446, 192, 46, 450, 194, 36, 449, 196, 60,

Generation output of midi files<br>

In [12]:
def generate_midi(tokenizer, generated_sequence, output_filename="rnn.mid"):
    try:
        vocab_size = tokenizer.vocab_size
        valid_sequence = [token for token in generated_sequence if 0 <= token < vocab_size]
        
        print(f"Original sequence length: {len(generated_sequence)}")
        print(f"Valid sequence length: {len(valid_sequence)}")
        
        if len(valid_sequence) < 2:
            print("Sequence too short or no valid tokens found")
            return None
            
        # Fix: Wrap the sequence in another list
        output_score = tokenizer.decode([valid_sequence])
        
        if len(output_score.tracks) == 0:
            print("Generated MIDI has no tracks")
            return None
            
        output_score.dump_midi(output_filename)
        print(f"Successfully generated {output_filename}")
        return output_score
        
    except Exception as e:
        print(f"Error during MIDI generation: {e}")
        print(f"Sequence sample: {generated_sequence[:20]}...")
        return None


# Usage
start_token = tokenizer.special_tokens_ids[1] if len(tokenizer.special_tokens_ids) > 1 else 1
generated_sequence = sample_top_k(model, start_token, max_length=1024)

# Generate MIDI safely
output_score = generate_midi(tokenizer, generated_sequence, "rnn.mid")


Original sequence length: 1025
Valid sequence length: 1025
Successfully generated rnn.mid
